In [98]:
pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 KB 3.2 MB/s eta 0:00:003.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 KB 2.5 MB/s eta 0:00:002.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Processamento do Texto

Obtenção e processamento dos textos que serão usados para a construção do grafo

In [10]:
import wikipedia
wikipedia.set_lang('en')

pages_search = [ "Anxiety", "Psychological_stress", "Depression_(mood)", "Bipolar_disorder", 
                "Post-traumatic_stress_disorder", "Eating_disorder", "Neurodevelopmental_disorder", "Mental_health" ]

pages_content = []

for page_title in pages_search:
    page = wikipedia.page(title=page_title, auto_suggest=False)
    pages_content.append(page.content.lower())

text = "\n\n".join(pages_content)
len(text)

375196

Divisão do texto em parte menores para realizar o processamento de entidades

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    separators=["\n\n","\n"]
)

chunks = text_splitter.split_text(text)
len(chunks)

1037

In [13]:
len(chunks[0])

397

In [14]:
chunks[2]

'\nthe emotion of anxiety can persist beyond the developmentally appropriate time-periods in response to specific events, and thus turning into one of the multiple anxiety disorders (e.g. generalized anxiety disorder, panic disorder). the difference between anxiety disorder (as mental disorder) and anxiety (as normal emotion), is that people with an anxiety disorder experience anxiety excessively or persistently during approximately 6 months, or even during shorter time-periods in children. anxiety disorders are among the most persistent mental problems and often last decades. anxiety can also be experienced within other mental disorders, e.g., obsessive-compulsive disorder, post-traumatic stress disorder.'

Carregando o modelo que será usado para NER (Named Entity Recognition)

In [9]:
from gliner import GLiNER
from tqdm import tqdm

model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.safetensors:  86%|########5 | 668M/781M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/home/gabriel/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Definição dos labels que serão buscados no texto

In [21]:
labels = [ "symptom", "treatment", "disease", "feeling", "risk factor", "behavior" ]

In [39]:
chunks_entities = []
chunks_labels = []
entity_list = []
duplicates = set()

for text_chunk in tqdm(chunks):
    entities = model.predict_entities(text_chunk, labels)
    chunk_entities = set()
    chunk_labels = set()
    for entity in entities:
        chunk_entities.add((entity["text"],entity["label"]))
        chunk_labels.add(entity["label"])
        if entity["text"] in duplicates:
            continue
        duplicates.add(entity["text"])
        entity_list.append((entity["text"], "=>", entity["label"]))
    chunks_entities.append(list(chunk_entities))
    chunks_labels.append(list(chunk_labels))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1037/1037 [02:07<00:00,  8.13it/s]


In [23]:
tratamentos = []
doencas = []
sentimentos = []
sintomas = []
fatores = []
comportamentos = []

for entity in entity_list:
    subject, p, obj = entity

    if obj == "treatment":
        tratamentos.append(subject.lower())
    elif obj == "disease":
        doencas.append(subject.lower())
    elif obj == "feeling":
        sentimentos.append(subject.lower())
    elif obj == "symptom":
        sintomas.append(subject.lower())
    elif obj == "risk factor":
        fatores.append(subject.lower())
    elif obj == "behavior":
        comportamentos.append(subject.lower())

print(f'Tratamento: {len(tratamentos)}')
print(f'Doenças: {len(doencas)}')
print(f'Sentimentos: {len(sentimentos)}')
print(f'Sintomas: {len(sintomas)}')
print(f'Fatores de Risco: {len(fatores)}')
print(f'Comportamentos: {len(comportamentos)}')

Tratamento: 260
Doenças: 250
Sentimentos: 263
Sintomas: 233
Fatores de Risco: 33
Comportamentos: 296


In [38]:
entity_list[:10]

[('anxiety', '=>', 'feeling'),
 ('fear', '=>', 'feeling'),
 ('nervous behavior', '=>', 'behavior'),
 ('pacing back and forth', '=>', 'behavior'),
 ('somatic complaints', '=>', 'symptom'),
 ('rumination', '=>', 'behavior'),
 ('muscular tension', '=>', 'symptom'),
 ('restlessness', '=>', 'symptom'),
 ('fatigue', '=>', 'symptom'),
 ("inability to catch one's breath", '=>', 'symptom')]

In [46]:
def format_entities(entities):
    return "\n".join(e for e in entities)

def format_entities_with_labels(entities):
    return "\n".join(": ".join([e[0],e[1]]) for e in entities)

In [42]:
"\n".join(":".join([e[0],e[1]]) for e in chunks_entities[i])

'increase or decrease in food intake:behavior\nwithdrawal from situations:behavior\nfoot tapping:behavior\nchanges in sleeping patterns:behavior\nincreased motor tension:behavior\nnegative feelings:feeling\nanxiety:symptom\nchanges in habits:behavior'

In [145]:
i = 10
entity = format_entities_with_labels(chunks_entities[i])
text_relations = chunks[i]

print(entity)
print(text_relations)

increase or decrease in food intake: behavior
withdrawal from situations: behavior
foot tapping: behavior
changes in sleeping patterns: behavior
increased motor tension: behavior
negative feelings: feeling
anxiety: symptom
changes in habits: behavior

the behavioral effects of anxiety may include withdrawal from situations which have provoked anxiety or negative feelings in the past. other effects may include changes in sleeping patterns, changes in habits, increase or decrease in food intake, and increased motor tension (such as foot tapping).


# Relation Entities

disease related disease 
disease related symptom 
disease related behavior
disease related risk factor
disease related treatment
disease related feeling
behavior related feeling
behavior related symptom
behavior related risk factor
symptom related feeling



In [144]:
system_message = """Extract all relationships between the following entities ONLY based on the provided text using relationships by tag in entity.
Return a list of JSON objects, for example:

<Examples>
[{{ "subject": "anxiety", "relationship": "related", "object": "fear" }},
{{ "subject": "stress", "relationship: "related", "object": "depression" }}]
</Examples>

- ONLY return triples and nothing else. None of "subject", "relationship" and "object" can be empty.

Relationships:

disease_related_disease 
disease_related_symptom 
disease_related_behavior
disease_related_risk factor
disease_related_treatment
disease_related_feeling
behavior_related_feeling
behavior_related_symptom
behavior_related_risk_factor
symptom_related_feeling

Entities: \n\n{entities}
"""

user_message = """Context: {text}\n\nTriples:"""

In [146]:
print(system_message.format(entities=entity))

Extract all relationships between the following entities ONLY based on the provided text using relationships by tag in entity.
Return a list of JSON objects, for example:

<Examples>
[{ "subject": "anxiety", "relationship": "related", "object": "fear" },
{ "subject": "stress", "relationship: "related", "object": "depression" }]
</Examples>

- ONLY return triples and nothing else. None of "subject", "relationship" and "object" can be empty.

Relationships:

disease_related_disease 
disease_related_symptom 
disease_related_behavior
disease_related_risk factor
disease_related_treatment
disease_related_feeling
behavior_related_feeling
behavior_related_symptom
behavior_related_risk_factor
symptom_related_feeling

Entities: 

increase or decrease in food intake: behavior
withdrawal from situations: behavior
foot tapping: behavior
changes in sleeping patterns: behavior
increased motor tension: behavior
negative feelings: feeling
anxiety: symptom
changes in habits: behavior



In [147]:
from openai import OpenAI
import json
client = OpenAI(
    organization='',
    project='',
    api_key='',
)


completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_message.format(entities=entity)},
    {"role": "user", "content": user_message.format(text=text_relations)}
  ]
)

triples = json.loads(completion.choices[0].message.content)
triples

[{'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'withdrawal from situations'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'changes in sleeping patterns'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'changes in habits'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'increase or decrease in food intake'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'increased motor tension'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_feeling',
  'object': 'negative feelings'}]

In [55]:
import time

errors = []
all_triples = []

for i in tqdm(range(len(chunks_entities))):
    try:
        
        entity = format_entities_with_labels(chunks_entities[i])
        text_relations = chunks[i]

        completion = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "system", "content": system_message.format(entities=entity)},
            {"role": "user", "content": user_message.format(text=text_relations)}
          ]
        )
        
        triples = json.loads(completion.choices[0].message.content)

        all_triples.append(triples)
        time.sleep(3)
        
    except Exception as e:
        print(f'Error chunk {i}: {e}')
        errors.append(completion.choices[0].message.content)
        all_triples.append([])


  1%|█                                                                                                                                           | 8/1037 [00:36<1:01:47,  3.60s/it]

Error chunk 7: Expecting value: line 1 column 1 (char 0)


  2%|██▌                                                                                                                                        | 19/1037 [01:26<1:05:24,  3.86s/it]

Error chunk 18: Expecting value: line 1 column 1 (char 0)


  4%|██████                                                                                                                                     | 45/1037 [03:22<1:08:29,  4.14s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 1030/1037 [1:09:16<00:24,  3.47s/it]

Error chunk 1029: Expecting value: line 1 column 1 (char 0)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 1034/1037 [1:09:29<00:08,  2.85s/it]

Error chunk 1033: Expecting value: line 1 column 1 (char 0)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1037/1037 [1:09:40<00:00,  4.03s/it]


In [56]:
len(all_triples)

1037

In [57]:
output_json = "../data/triples.json"

json_data = json.dumps(all_triples,indent=4)

with open(output_json,"w") as file:
    file.write(json_data)

In [114]:
import json

input_json = "../data/triples.json"

with open(input_json,"r") as file:
    all_triples = json.load(file)

all_triples[0]

[{'subject': 'anxiety',
  'relationship': 'disease_related_feeling',
  'object': 'dread'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'nervous behavior'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'pacing back and forth'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_symptom',
  'object': 'somatic complaints'},
 {'subject': 'anxiety',
  'relationship': 'disease_related_behavior',
  'object': 'rumination'},
 {'subject': 'fear',
  'relationship': 'disease_related_feeling',
  'object': 'dread'}]

In [142]:
def get_color(n):
    type_to_color = {
        "symptom": "#fcce03", 
        "treatment": "#62fc03", 
        "disease": "#fc0303", 
        "feeling": "#0b03fc",
        "risk factor": "#fc7303",
        "behavior": "#03f4fc",
    }

    if n.lower() in sintomas:
        return type_to_color["symptom"]
    if n.lower() in sentimentos:
        return type_to_color["feeling"]
    if n.lower() in tratamentos:
        return type_to_color["treatment"]
    if n.lower() in doencas:
        return type_to_color["disease"]
    if n.lower() in fatores:
        return type_to_color["risk factor"]
    if n.lower() in comportamentos:
        return type_to_color["behavior"]

    return "purple" 

def get_size(n):
    type_to_size = {
        "disease": 50, 
        "risk factor": 40,
        "symptom": 30,
        "treatment": 20, 
        "feeling": 15,
        "behavior": 10
    }

    if n.lower() in sintomas:
        return type_to_size["symptom"]
    if n.lower() in sentimentos:
        return type_to_size["feeling"]
    if n.lower() in tratamentos:
        return type_to_size["treatment"]
    if n.lower() in doencas:
        return type_to_size["disease"]
    if n.lower() in fatores:
        return type_to_size["risk factor"]
    if n.lower() in comportamentos:
        return type_to_size["behavior"]

    return 5

In [59]:
from pyvis.network import Network
import networkx as nx


graph = nx.Graph()

for items in all_triples:
    for item in items:
        try:
            node_1 = item["subject"]
            node_2 = item["object"]
    
            graph.add_node(node_1, title=node_1, color=get_color(node_1), size=get_size(node_1), label=node_1)
            graph.add_node(node_2, title=node_2, color=get_color(node_2), size=get_size(node_2), label=node_2)
            graph.add_edge(node_1, node_2, title=item["relationship"])
        except Exception as e:
            print(e)
            print(f'Error in item: {item}')

string indices must be integers
Error in item: subject
string indices must be integers
Error in item: relationship
string indices must be integers
Error in item: object


In [60]:
len(graph.nodes)

3044

In [61]:
len(graph.edges)

3584

In [69]:
nt = Network(height="750px", width="100%")

nt.from_nx(graph)
nt.force_atlas_2based(central_gravity=0.015, gravity=-60)
nt.show_buttons(filter_=['physics'])
nt.show("../data/graph.html",notebook=False)

../data/graph.html
Opening in existing browser session.


In [71]:
from IPython.display import IFrame
IFrame("../data/graph.html", width=1000, height=800)

In [95]:
no_entities = []

for node in list(graph.nodes):
    no_entity = False
    for entity in entity_list:
        if entity[0] == node:
            no_entity = False
            break
        else:
            no_entity = True
    
    if no_entity:
        no_entities.append(node)

len(no_entities)

1893

In [105]:
for no_entity in no_entities:
    try:
        graph.remove_node(no_entity)
    except Exception as e:
        continue
print(f'Nodes: {graph.number_of_nodes()}')
print(f'Edges: {graph.number_of_edges()}')

Nodes: 1151
Edges: 1338


In [100]:
nt = Network(height="750px", width="100%")

nt.from_nx(graph)
nt.force_atlas_2based(central_gravity=0.015, gravity=-60)
nt.show_buttons(filter_=['physics'])
nt.show("../data/graph-clear.html",notebook=False)

../data/graph-clear.html
Opening in existing browser session.


In [102]:
from IPython.display import IFrame
IFrame("../data/graph-clear.html", width=1000, height=800)

In [106]:
title_edges = nx.get_edge_attributes(graph,"title")
title_edges

{('anxiety', 'dread'): 'related',
 ('anxiety', 'nervous behavior'): 'disease_related_behavior',
 ('anxiety', 'pacing back and forth'): 'disease_related_behavior',
 ('anxiety', 'somatic complaints'): 'disease_related_symptom',
 ('anxiety', 'rumination'): 'disease_related_behavior',
 ('anxiety', 'fear'): 'disease_related_symptom',
 ('anxiety', 'muscular tension'): 'disease_related_symptom',
 ('anxiety', 'restlessness'): 'disease_related_feeling',
 ('anxiety', 'fatigue'): 'disease_related_symptom',
 ('anxiety', "inability to catch one's breath"): 'disease_related_symptom',
 ('anxiety', 'tightness in the abdominal region'): 'disease_related_symptom',
 ('anxiety', 'nausea'): 'disease_related_symptom',
 ('anxiety', 'problems in concentration'): 'disease_related_symptom',
 ('anxiety', 'generalized anxiety disorder'): 'disease_related_disease',
 ('anxiety', 'panic disorder'): 'disease_related_symptom',
 ('anxiety', 'obsessive-compulsive disorder'): 'disease_related_disease',
 ('anxiety', 'post

- Número de Ocorrências
- Probabilidade
- Especificidade
- Confiabilidade

In [109]:
relationships = [
    "disease_related_disease",
    "disease_related_symptom",
    "disease_related_behavior",
    "disease_related_risk factor",
    "disease_related_treatment",
    "disease_related_feeling",
    "behavior_related_feeling",
    "behavior_related_symptom",
    "behavior_related_risk factor",
    "symptom_related_feeling"
]

len(relationships)

10

In [123]:
entities_list = [] 

for e in entity_list:
    entities_list.append(e[0])
    
entities_list[:10]

['anxiety',
 'fear',
 'nervous behavior',
 'pacing back and forth',
 'somatic complaints',
 'rumination',
 'muscular tension',
 'restlessness',
 'fatigue',
 "inability to catch one's breath"]

In [127]:
triples_clear = []

for triples in all_triples:
    for triple in triples:
        try:
            subject = triple['subject']
            predicate = triple["relationship"]
            obj = triple['object']
            
            if predicate in relationships and subject in entities_list and obj in entities_list:
                triples_clear.append((subject,predicate,obj))
        except Exception as e:
            print(e)
            continue
            
len(triples_clear)

string indices must be integers
string indices must be integers
string indices must be integers


1202

In [128]:
triples_clear[:10]

[('anxiety', 'disease_related_feeling', 'dread'),
 ('anxiety', 'disease_related_behavior', 'nervous behavior'),
 ('anxiety', 'disease_related_behavior', 'pacing back and forth'),
 ('anxiety', 'disease_related_symptom', 'somatic complaints'),
 ('anxiety', 'disease_related_behavior', 'rumination'),
 ('fear', 'disease_related_feeling', 'dread'),
 ('anxiety', 'disease_related_feeling', 'fear'),
 ('anxiety', 'disease_related_symptom', 'muscular tension'),
 ('anxiety', 'disease_related_symptom', 'restlessness'),
 ('anxiety', 'disease_related_symptom', 'fatigue')]

In [131]:
import pandas as pd

data = pd.DataFrame(
    triples_clear,
    columns=["subject", "predicate", "object"]
)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   subject    1202 non-null   object
 1   predicate  1202 non-null   object
 2   object     1202 non-null   object
dtypes: object(3)
memory usage: 28.3+ KB


In [138]:
data["subject"].value_counts()

subject
bipolar disorder                  169
ptsd                               87
anxiety                            73
eating disorders                   51
depression                         34
                                 ... 
re-experiencing                     1
heightened sense of threat          1
single-session debriefing           1
multiple session interventions      1
hiv/aids                            1
Name: count, Length: 327, dtype: int64

In [141]:
data["predicate"].value_counts()

predicate
disease_related_behavior        273
disease_related_treatment       250
disease_related_symptom         231
disease_related_disease         178
disease_related_feeling         129
disease_related_risk factor      71
behavior_related_feeling         28
symptom_related_feeling          21
behavior_related_symptom         15
behavior_related_risk factor      6
Name: count, dtype: int64

In [143]:
graph = nx.Graph()

for item in triples_clear:
    try:
        node_1 = item[0]
        node_2 = item[2]

        graph.add_node(node_1, title=node_1, color=get_color(node_1), size=get_size(node_1), label=node_1)
        graph.add_node(node_2, title=node_2, color=get_color(node_2), size=get_size(node_2), label=node_2)
        graph.add_edge(node_1, node_2, title=item[1])
    except Exception as e:
        print(e)
        print(f'Error in item: {item}')

nt = Network(height="750px", width="100%")

nt.from_nx(graph)
nt.force_atlas_2based(central_gravity=0.015, gravity=-60)
nt.show_buttons(filter_=['physics'])
nt.show("../data/graph-clear-entities.html",notebook=False)

../data/graph-clear-entities.html


Opening in existing browser session.


In [2]:
from IPython.display import IFrame
IFrame("../data/graph-clear-entities.html", width=1000, height=800)

In [151]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

In [148]:
localy = nx.degree_centrality(graph)
localy

{'anxiety': 0.14520202020202022,
 'dread': 0.0025252525252525255,
 'nervous behavior': 0.0012626262626262627,
 'pacing back and forth': 0.0012626262626262627,
 'somatic complaints': 0.0012626262626262627,
 'rumination': 0.0012626262626262627,
 'fear': 0.012626262626262628,
 'muscular tension': 0.0012626262626262627,
 'restlessness': 0.0025252525252525255,
 'fatigue': 0.006313131313131314,
 "inability to catch one's breath": 0.0012626262626262627,
 'tightness in the abdominal region': 0.0012626262626262627,
 'nausea': 0.003787878787878788,
 'problems in concentration': 0.0012626262626262627,
 'generalized anxiety disorder': 0.0025252525252525255,
 'panic disorder': 0.006313131313131314,
 'obsessive-compulsive disorder': 0.0025252525252525255,
 'post-traumatic stress disorder': 0.012626262626262628,
 'mental disorder': 0.010101010101010102,
 'mental disorders': 0.007575757575757576,
 'depression': 0.06818181818181819,
 'self-harm': 0.005050505050505051,
 'suicide': 0.012626262626262628,


In [166]:

def sortTuple(node):
    return node[1]


In [167]:
localy_sort = []

for node in localy:
    localy_sort.append((node,localy[node]))

localy_sort.sort(reverse=True, key=sortTuple)

localy_sort

[('bipolar disorder', 0.1792929292929293),
 ('anxiety', 0.14520202020202022),
 ('ptsd', 0.14520202020202022),
 ('eating disorders', 0.08080808080808081),
 ('depression', 0.06818181818181819),
 ('anorexia nervosa', 0.042929292929292935),
 ('stress', 0.04040404040404041),
 ('depressed mood', 0.030303030303030304),
 ('anxiety disorders', 0.026515151515151516),
 ('bulimia nervosa', 0.025252525252525256),
 ('anorexia', 0.02398989898989899),
 ('mania', 0.01893939393939394),
 ('eating disorder', 0.01893939393939394),
 ('mental illness', 0.015151515151515152),
 ('bulimia', 0.01388888888888889),
 ('fear', 0.012626262626262628),
 ('post-traumatic stress disorder', 0.012626262626262628),
 ('suicide', 0.012626262626262628),
 ('prevention', 0.012626262626262628),
 ('treatment', 0.012626262626262628),
 ('antidepressants', 0.012626262626262628),
 ('adhd', 0.012626262626262628),
 ('neurodevelopmental disorders', 0.012626262626262628),
 ('depressive episodes', 0.011363636363636364),
 ('mental disorder'

In [149]:
closeness = nx.closeness_centrality(graph)
closeness

{'anxiety': 0.3729490022172949,
 'dread': 0.263069403714565,
 'nervous behavior': 0.26194276815261824,
 'pacing back and forth': 0.26194276815261824,
 'somatic complaints': 0.26194276815261824,
 'rumination': 0.26194276815261824,
 'fear': 0.2964790904684264,
 'muscular tension': 0.26194276815261824,
 'restlessness': 0.2795412996509889,
 'fatigue': 0.31592787377911347,
 "inability to catch one's breath": 0.26194276815261824,
 'tightness in the abdominal region': 0.26194276815261824,
 'nausea': 0.2645486001887386,
 'problems in concentration': 0.26194276815261824,
 'generalized anxiety disorder': 0.2631826005319981,
 'panic disorder': 0.3015958400573785,
 'obsessive-compulsive disorder': 0.2684970867587198,
 'post-traumatic stress disorder': 0.2642057726290988,
 'mental disorder': 0.26409169414350764,
 'mental disorders': 0.28621261751818605,
 'depression': 0.3743184599977746,
 'self-harm': 0.29748850371418467,
 'suicide': 0.32106895728942975,
 'withdrawal from situations': 0.26194276815

In [168]:
closeness_sort = []

for node in closeness:
    closeness_sort.append((node,closeness[node]))

closeness_sort.sort(reverse=True, key=sortTuple)

closeness_sort

[('bipolar disorder', 0.37523703290574456),
 ('depression', 0.3743184599977746),
 ('anxiety', 0.3729490022172949),
 ('stress', 0.3465361833633788),
 ('anxiety disorders', 0.34419604031309153),
 ('ptsd', 0.3426534250063662),
 ('eating disorders', 0.33186997484338776),
 ('suicide', 0.32106895728942975),
 ('cognitive behavioral therapy', 0.3197262747707076),
 ('fatigue', 0.31592787377911347),
 ('antidepressants', 0.3146277590722035),
 ('substance abuse', 0.3115824572778215),
 ('treatment', 0.31016042780748665),
 ('medications', 0.3078190053529761),
 ('stroke', 0.3053601416057731),
 ('symptoms', 0.3033910533910534),
 ('emdr', 0.3027902790279028),
 ('major depressive disorder', 0.3018935654671094),
 ('panic disorder', 0.3015958400573785),
 ('cbt', 0.2996748474455481),
 ('euphoria', 0.29908868637474995),
 ('pharmacotherapy', 0.2987964648931918),
 ('multiple sclerosis', 0.29850481387816674),
 ('medication', 0.2977781711958927),
 ('self-harm', 0.29748850371418467),
 ('obsessive–compulsive diso

In [150]:
betweenness = nx.betweenness_centrality(graph)
betweenness

{'anxiety': 0.24474662651722404,
 'dread': 0.0,
 'nervous behavior': 0.0,
 'pacing back and forth': 0.0,
 'somatic complaints': 0.0,
 'rumination': 0.0,
 'fear': 0.018304918687675392,
 'muscular tension': 0.0,
 'restlessness': 0.0,
 'fatigue': 0.00462580957909509,
 "inability to catch one's breath": 0.0,
 'tightness in the abdominal region': 0.0,
 'nausea': 0.019083092902194152,
 'problems in concentration': 0.0,
 'generalized anxiety disorder': 8.391549354273946e-05,
 'panic disorder': 0.0033977188185387985,
 'obsessive-compulsive disorder': 0.0030480429342629083,
 'post-traumatic stress disorder': 0.018997330946646518,
 'mental disorder': 0.006378108606821455,
 'mental disorders': 0.006931882058642351,
 'depression': 0.14037035480047205,
 'self-harm': 0.0006634666177063375,
 'suicide': 0.027095607207553985,
 'withdrawal from situations': 0.0,
 'negative feelings': 0.0,
 'changes in sleeping patterns': 0.0,
 'changes in habits': 0.0,
 'increase or decrease in food intake': 0.0,
 'incr

In [169]:
betweenness_sort = []

for node in betweenness:
    betweenness_sort.append((node,betweenness[node]))

betweenness_sort.sort(reverse=True, key=sortTuple)

betweenness_sort

[('bipolar disorder', 0.3256371640339387),
 ('anxiety', 0.24474662651722404),
 ('ptsd', 0.22926652204764203),
 ('eating disorders', 0.14633982676852156),
 ('depression', 0.14037035480047205),
 ('stress', 0.06446388144823373),
 ('anorexia nervosa', 0.06365244974628288),
 ('anxiety disorders', 0.04251832134306258),
 ('treatment', 0.03875346314661797),
 ('diabetes mellitus', 0.034734194026229424),
 ('neurodevelopmental disorders', 0.03287936252538023),
 ('anorexia', 0.031224526098917925),
 ('mania', 0.03075017761053322),
 ('suicide', 0.027095607207553985),
 ('bulimia nervosa', 0.024601553615172553),
 ('impulsivity', 0.02190042013050863),
 ('depressed mood', 0.021212543015405656),
 ('adhd', 0.01983701319984506),
 ('nausea', 0.019083092902194152),
 ('post-traumatic stress disorder', 0.018997330946646518),
 ('fear', 0.018304918687675392),
 ('eating disorder', 0.017415381238437642),
 ('celiac disease', 0.01728366125220781),
 ('bulimia', 0.014730842407904142),
 ('phenylketonuria', 0.0132456039